In [17]:
import json
import csv
import pandas as pd
import gurobipy as gp
from datetime import datetime

In [2]:
OBP_AVG = 0.327
SLG_AVG = 0.365

FPCT_AVG = {
    '2': 0.992,
    '3': 0.993,
    '4': 0.977, 
    '5': 0.929,
    '6': 0.964,
    '7': 0.977,
    '8': 0.988,
    '9': 0.976
}

POSITION_WEIGHT = {
    '2': {"batting": 0.95, "fielding": 1.05},
    '3': {"batting": 1.21, "fielding": 0.83},
    '4': {"batting": 0.97, "fielding": 1.03},
    '5': {"batting": 0.96, "fielding": 1.04},
    '6': {"batting": 0.94, "fielding": 1.06},
    '7': {"batting": 1.02, "fielding": 0.98},
    '8': {"batting": 1.04, "fielding": 0.96},
    '9': {"batting": 1.17, "fielding": 0.85},
}

OPPONENT_WEIGHT = {
    "brothers": 1.15,
    "lions": 1.11,
    "monkeys": 0.96, 
    "guardians": 0.93,
    "dragons": 0.85
}

dataRoot = "../clean/clean_all_"
playerRoot = "../球員異動/"

teamNameMap = {
    "中信兄弟": "brothers", 
    "味全龍": "dragons",
    "富邦悍將": "guardians", 
    "統一7-ELEVEn獅": "lions",
    "樂天桃猿": "monkeys"
}

### Import Data

In [ ]:
class Player:
    def __init__(self, name, data):
        self.name = name
        self.data = data

        # A_{ij}
        canPlay = []
        for i in range(9):
            if (i + 1) in data["fielding"]["pos"]:
                canPlay.append(1)
            else:
                canPlay.append(0)
        self.canPlay = canPlay

        # F_{ij}
        F = {}
        for i in data["fielding"]["pos"]:
            F[str(i)] = float(data["fielding"]["FPCT"][str(i)]) / FPCT_AVG[str(i)]
            if F[str(i)] == 1:
                F[str(i)] = 0.98
            if F[str(i)] < 0.8:
                F[str(i)] = 0.8
        self.F = F

In [ ]:
allData = {}
for team in teamNameMap.keys():
    with open(dataRoot + team + ".json", encoding="utf-8") as f:
        temp = []
        for line in f:
            player = json.loads(line)
            temp.append(Player(player["name"], player["data"]))
        allData[teamNameMap[team]] = temp

brothers = allData["brothers"]
dragons = allData["dragons"]
guardians = allData["guardians"]
lions = allData["lions"]
monkeys = allData["monkeys"]

In [39]:
playerList = {}

for team in teamNameMap.keys():
    tempDict = pd.read_csv(playerRoot + team + ".csv", header=0, index_col=0).to_dict()
    for key in tempDict.keys():
        tempDict[key] = list(filter(lambda d: pd.isna(d) == False, list(tempDict[key].values())))
        key = datetime.strptime(key, "%Y-%m-%d").date()
    playerList[teamNameMap[team]] = tempDict

In [37]:
def createDate(mm, dd):
    return datetime(2022, mm, dd, 0, 0, 0).date()

### Self-defined Parameters

In [ ]:
aTeam = "中信兄弟"
aMonth = "May"
aDate = [createDate(5, 3), createDate(5, 5), createDate(5, 6), createDate(5, 7), createDate(5, 8)]
aField = ["新莊", "澄清湖", "洲際", "洲際", "洲際"]
aOppo = ["富邦悍將", "味全龍", "統一7-ELEVEn獅", "統一7-ELEVEn獅", "統一7-ELEVEn獅"]
aPitcher = ["陳鴻文", "王維中", "江辰晏", "李其峰", "邱浩鈞"]
aStar = ["許基宏", "江坤宇"]
aPotential = ["岳東華"]
aInjured = ["周思齊"]

### Optimization

In [ ]:
# model = gp.Model("model")
# for i in range()
# x.append(p1.addVar(lb=0, vtype="C", name="x" + str(i)))
